In [1]:
import pandas as pd
import numpy as np

In [143]:
def check_revealed_preference(price_a, quantity_a, price_b, quantity_b):
    """
    Detect whether two consumption bundles violate WARP.

    :param price_a: Price vector of the first consumption bundle
    :param quantity_a: Quantity vector of the first consumption bundle
    :param price_b: Price vector of the second consumption bundle
    :param quantity_b: Quantity vector of the second consumption bundle
    :return: True if WARP is satisfied, False otherwise
    """
    expenditure_a_a = np.dot(price_a, quantity_a)
    expenditure_a_b = np.dot(price_a, quantity_b)

    expenditure_b_b = np.dot(price_b, quantity_b)
    expenditure_b_a = np.dot(price_b, quantity_a)

    print(f"Expenditures: A->A: {expenditure_a_a}, A->B: {expenditure_a_b}, B->B: {expenditure_b_b}, B->A: {expenditure_b_a}")

    # Check for violations of WARP
    if expenditure_a_b <= expenditure_a_a and expenditure_b_a <= expenditure_b_b:
        return False  # Violates WARP
    return True  # Satisfies WARP

In [161]:
def check_strict_direct_preference(price_a, quantity_a, price_b, quantity_b):
    """
    Check if there is a strict direct revealed preference (s.d.r.p.) between two consumption bundles.

    :param price_a: Price vector of the first consumption bundle
    :param quantity_a: Quantity vector of the first consumption bundle
    :param price_b: Price vector of the second consumption bundle
    :param quantity_b: Quantity vector of the second consumption bundle
    :return: True if A strictly directly reveals preference over B, False otherwise
    """
    expenditure_a_a = np.dot(price_a, quantity_a)
    expenditure_a_b = np.dot(price_a, quantity_b)

    expenditure_b_b = np.dot(price_b, quantity_b)
    expenditure_b_a = np.dot(price_b, quantity_a)

    # Check for strict direct revealed preference
    if expenditure_a_b < expenditure_a_a or expenditure_b_a < expenditure_b_b:
        return True  # A s.d.r.p. B
    return False  # No strict direct preference

In [124]:
def detect_cycle(preference_graph):
    """
    检测偏好图中是否存在循环偏好。
    Detect whether there is a cycle in the preference graph.

    - 输入 (Inputs):
        preference_graph: 偏好关系图，键为节点，值为其直接偏好的节点列表
                          Preference graph, where keys are nodes and values are lists of directly preferred nodes
    - 返回 (Returns):
        True: 如果存在循环偏好 / If a cycle is detected in preferences
        False: 如果不存在循环偏好 / If no cycle exists in preferences
    """
    def dfs(node, visited, stack):
        visited.add(node)
        stack.add(node)

        for neighbor in preference_graph.get(node, []):
            if neighbor not in visited:
                if dfs(neighbor, visited, stack):
                    return True
            elif neighbor in stack:
                return True

        stack.remove(node)
        return False

    visited = set()
    for node in preference_graph:
        if node not in visited:
            if dfs(node, visited, set()):
                return True
    return False

In [145]:
def check_warp_violation(quantity_list, price_list):
    """
    检测是否存在 WARP 违背行为。
    Check if there are any violations of the Weak Axiom of Revealed Preference (WARP).

    - 输入 (Inputs):
        quantity_list: 每组消费组合的数量向量链表 / A list of quantity vectors for each consumption bundle
        price_list: 每组消费组合的价格向量链表 / A list of price vectors for each consumption bundle
    - 返回 (Returns):
        结果字典，包含以下内容 / A dictionary containing the following:
            - 'has_violation': 是否存在任何WARP违背 / Whether any WARP violations exist
            - 'pair_violation': 存在违背的消费组合对及其状态 / Pairs of consumption bundles where violations occur
            - 'violation_count': 违背的组合对数量 / Number of violating pairs
            - 'violation_ratio': 违背比例 / Proportion of violating pairs
    """
    if len(quantity_list) != len(price_list):
        raise ValueError("Quantity list and price list must have the same length.")

    n_rounds = len(quantity_list)
    pair_violation = []
    total_violation_count = 0
    has_violation = False

    for i in range(n_rounds):
        quantity_a = quantity_list[i]
        price_a = price_list[i]

        for j in range(i + 1, n_rounds):
            quantity_b = quantity_list[j]
            price_b = price_list[j]

            if not (len(price_a) == len(price_b) == len(quantity_a) == len(quantity_b)):
                raise ValueError("Price and quantity vectors must be of the same length for all rounds.")

            if not all(isinstance(x, (int, float)) for x in price_a + price_b + quantity_a + quantity_b):
                raise ValueError("Price and quantity vectors must contain only numeric values.")

            rp_flag = check_revealed_preference(price_a, quantity_a, price_b, quantity_b)

            if not rp_flag:
                pair_violation.append({
                    'round_pair': (i, j),
                    'violation': True
                })
                has_violation = True
                total_violation_count += 1

    total_comparisons = n_rounds * (n_rounds - 1) / 2
    violation_ratio = total_violation_count / total_comparisons if total_comparisons > 0 else 0

    return {
        'has_violation': has_violation,
        'pair_violation': pair_violation,
        'violation_count': total_violation_count,
        'violation_ratio': violation_ratio
    }

In [147]:
def check_sarp_violation(quantity_list, price_list):
    """
    检测是否存在 SARP 违背行为。
    Check if there are any violations of the Strong Axiom of Revealed Preference (SARP).

    - 输入 (Inputs):
        quantity_list: 每组消费组合的数量向量链表 / A list of quantity vectors for each consumption bundle
        price_list: 每组消费组合的价格向量链表 / A list of price vectors for each consumption bundle
    - 返回 (Returns):
        结果字典，包含以下内容 / A dictionary containing the following:
            - 'has_violation': 是否存在任何SARP违背 / Whether any SARP violations exist
            - 'violation_cycles': 偏好循环 / List of preference cycles
    """
    if len(quantity_list) != len(price_list):
        raise ValueError("Quantity list and price list must have the same length.")

    n_rounds = len(quantity_list)
    preference_graph = {}
    pair_violation = []
    total_violation_count = 0
    has_violation = False

    for i in range(n_rounds):
        quantity_a = quantity_list[i]
        price_a = price_list[i]

        for j in range(n_rounds):
            if i == j:
                continue

            quantity_b = quantity_list[j]
            price_b = price_list[j]

            if not (len(price_a) == len(price_b) == len(quantity_a) == len(quantity_b)):
                raise ValueError("Price and quantity vectors must be of the same length for all rounds.")

            if not all(isinstance(x, (int, float)) for x in price_a + price_b + quantity_a + quantity_b):
                raise ValueError("Price and quantity vectors must contain only numeric values.")

            rp_flag = check_revealed_preference(price_a, quantity_a, price_b, quantity_b)

            if rp_flag:
                preference_graph.setdefault(i, []).append(j)
            else:
                pair_violation.append({
                    'round_pair': (i, j),
                    'violation': True
                })
                has_violation = True
                total_violation_count += 1

    total_comparisons = n_rounds * (n_rounds - 1) / 2
    violation_ratio = total_violation_count / total_comparisons if total_comparisons > 0 else 0

    # 调用检测循环的函数 / Call the cycle detection function
    has_cycle_violation = detect_cycle(preference_graph)

    return {
        'has_violation': has_violation or has_cycle_violation,
        'pair_violation': pair_violation,
        'violation_cycles': preference_graph if has_cycle_violation else None,
        'violation_count': total_violation_count,
        'violation_ratio': violation_ratio
    }

In [183]:
def check_garp_violation(quantity_list, price_list):
    """
    Check if there are any violations of the Generalized Axiom of Revealed Preference (GARP).

    :param quantity_list: A list of quantity vectors for each consumption bundle
    :param price_list: A list of price vectors for each consumption bundle
    :return: A dictionary containing the following:
        - 'has_violation': Whether any GARP violations exist
        - 'pair_violation': Pairs of consumption bundles where violations occur
        - 'violation_count': Number of violating pairs
        - 'violation_ratio': Proportion of violating pairs
        - 'violation_cycles': Cycles in preference graph if any
    """
    if len(quantity_list) != len(price_list):
        raise ValueError("Quantity list and price list must have the same length.")

    n_rounds = len(quantity_list)
    preference_graph = {}
    pair_violation = []
    total_violation_count = 0
    has_violation = False

    for i in range(n_rounds):
        quantity_a = quantity_list[i]
        price_a = price_list[i]

        for j in range(n_rounds):
            if i == j:
                continue

            quantity_b = quantity_list[j]
            price_b = price_list[j]

            if not (len(price_a) == len(price_b) == len(quantity_a) == len(quantity_b)):
                raise ValueError("Price and quantity vectors must be of the same length for all rounds.")

            rp_flag = check_revealed_preference(price_a, quantity_a, price_b, quantity_b)

            # Check if B is strictly directly preferred to A (potential GARP violation)
            sdrp_flag = check_strict_direct_preference(price_b, quantity_b, price_a, quantity_a)

            if rp_flag and sdrp_flag:
                pair_violation.append({
                    'round_pair': (i, j),
                    'violation': True
                })
                has_violation = True
                total_violation_count += 1
            elif rp_flag:
                preference_graph.setdefault(i, []).append(j)
            else:
                pair_violation.append({
                    'round_pair': (i, j),
                    'violation': True
                })
                has_violation = True
                total_violation_count += 1

    total_comparisons = n_rounds * (n_rounds - 1)
    violation_ratio = total_violation_count / total_comparisons if total_comparisons > 0 else 0

    has_cycle_violation = detect_cycle(preference_graph)

    return {
        'has_violation': has_violation or has_cycle_violation,
        'pair_violation': pair_violation,
        'violation_cycles': preference_graph if has_cycle_violation else None,
        'violation_count': total_violation_count,
        'violation_ratio': violation_ratio
    }

In [184]:
# 示例使用
quantity_list = [
    [1, 0, 0],  # 第一组消费组合的数量向量
    [0, 1, 0],  # 第二组消费组合的数量向量
    [0, 0, 1]   # 第三组消费组合的数量向量
]

price_list = [
    [2, 2, 3],  # 第一组消费组合的价格向量
    [4, 2, 2],  # 第二组消费组合的价格向量
    [2, 4, 3]   # 第三组消费组合的价格向量
]

warp_violation_results = check_warp_violation(quantity_list, price_list)
print(warp_violation_results)

sarp_violation_results = check_sarp_violation(quantity_list, price_list)
print(sarp_violation_results)

garp_violation_results = check_garp_violation(quantity_list, price_list)
print(garp_violation_results)

Expenditures: A->A: 2, A->B: 2, B->B: 2, B->A: 4
Expenditures: A->A: 2, A->B: 3, B->B: 3, B->A: 2
Expenditures: A->A: 2, A->B: 2, B->B: 3, B->A: 4
{'has_violation': False, 'pair_violation': [], 'violation_count': 0, 'violation_ratio': 0.0}
Expenditures: A->A: 2, A->B: 2, B->B: 2, B->A: 4
Expenditures: A->A: 2, A->B: 3, B->B: 3, B->A: 2
Expenditures: A->A: 2, A->B: 4, B->B: 2, B->A: 2
Expenditures: A->A: 2, A->B: 2, B->B: 3, B->A: 4
Expenditures: A->A: 3, A->B: 2, B->B: 2, B->A: 3
Expenditures: A->A: 3, A->B: 4, B->B: 2, B->A: 2
{'has_violation': True, 'pair_violation': [], 'violation_cycles': {0: [1, 2], 1: [0, 2], 2: [0, 1]}, 'violation_count': 0, 'violation_ratio': 0.0}
Expenditures: A->A: 2, A->B: 2, B->B: 2, B->A: 4
Expenditures: A->A: 2, A->B: 3, B->B: 3, B->A: 2
Expenditures: A->A: 2, A->B: 4, B->B: 2, B->A: 2
Expenditures: A->A: 2, A->B: 2, B->B: 3, B->A: 4
Expenditures: A->A: 3, A->B: 2, B->B: 2, B->A: 3
Expenditures: A->A: 3, A->B: 4, B->B: 2, B->A: 2
{'has_violation': True, '